In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from pyntcloud import PyntCloud
from modelnet import * # Includes the dataloader + data augmentation functions
import utils
import time

/home/marcus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load ModelNet 40

In [2]:
# Load the ModelNet data loader
modelnet = ModelNet(shuffle=True)

## Define the network with tensorflow

![PointNet](PointNet-Architecture.png)

In [3]:
# Point cloud placeholder -> [batch_size x num_points x 3]
points = tf.placeholder(tf.float32, [None, modelnet.num_points, 3])
labels = tf.placeholder(tf.float32, [None, modelnet.num_categories])
batch_norm_decay = tf.placeholder(tf.float32)
dropout_rate = tf.placeholder(tf.float32)

# Initializes the bias to identity
def bias_identity_init(shape, dtype, partition_info):
    rows = np.sqrt(shape[0]).astype(dtype=np.int32)
    return tf.reshape(tf.eye(rows, dtype=dtype), [-1])

def transformation_network(inputs, output_size):
    # Increase num features -> MLP(64, 128, 1024)
    feat = tf.layers.conv1d(inputs, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu)   # [batch_size x num_points x 64]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    feat = tf.layers.conv1d(feat, filters=128, kernel_size=1, strides=1, activation=tf.nn.relu)  # [batch_size x num_points x 128]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    feat = tf.layers.conv1d(feat, filters=1024, kernel_size=1, strides=1, activation=tf.nn.relu) # [batch_size x num_points x 1024]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)

    # Extract global features -> maxpool
    feat = tf.reduce_max(feat, 1) # [batch_size x 1024]

    # Generate predictions -> FC(512, 256, num_categories)
    feat = tf.layers.dense(feat, units=512, activation=tf.nn.relu) # [batch_size x 512]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    feat = tf.layers.dense(feat, units=256, activation=tf.nn.relu) # [batch_size x 256]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    return tf.layers.dense(feat, units=output_size, activation=None,
                           bias_initializer=bias_identity_init,
                           kernel_initializer=tf.zeros_initializer) # [batch_size x output_size]

def final_transformation_network(inputs, output_size):
    # Increase num features -> MLP(64, 128, 1024)
    feat = tf.layers.conv1d(inputs, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu)   # [batch_size x num_points x 64]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    feat = tf.layers.conv1d(feat, filters=128, kernel_size=1, strides=1, activation=tf.nn.relu)  # [batch_size x num_points x 128]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    feat = tf.layers.conv1d(feat, filters=1024, kernel_size=1, strides=1, activation=tf.nn.relu) # [batch_size x num_points x 1024]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)

    # Extract global features -> maxpool
    feat = tf.reduce_max(feat, 1) # [batch_size x 1024]

    # Generate predictions -> FC(512, 256, num_categories)
    feat = tf.layers.dense(feat, units=512, activation=tf.nn.relu) # [batch_size x 512]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    feat = tf.layers.dense(feat, units=256, activation=tf.nn.relu) # [batch_size x 256]
    feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
    feat = tf.layers.dropout(feat, rate=dropout_rate) # Dropout with keep rate 0.7 = 1.0 - 0.3
    return tf.layers.dense(feat, units=output_size, activation=None) # [batch_size x output_size]

# Transform input
transformation_matrix = tf.reshape(transformation_network(points, 3 * 3), [-1, 3, 3])
feat = tf.matmul(points, transformation_matrix)

# Points to features -> MLP(64, 64)
feat = tf.layers.conv1d(feat, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu) # [batch_size x num_points x 64]
feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)
feat = tf.layers.conv1d(feat, filters=64, kernel_size=1, strides=1, activation=tf.nn.relu) # [batch_size x num_points x 64]
feat = tf.contrib.layers.batch_norm(feat, decay=batch_norm_decay)

# Transform features
transformation_matrix = tf.reshape(transformation_network(feat, 64 * 64), [-1, 64, 64])

# Add Frobenious norm of feature transform matrix to loss
Lreg = tf.nn.l2_loss(tf.eye(64) - tf.matmul(transformation_matrix, tf.transpose(transformation_matrix, perm=[0, 2, 1])))
feat = tf.matmul(feat, transformation_matrix)

# Features to classification
feat = final_transformation_network(feat, modelnet.num_categories)

# Cross entropy softmax loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=feat, labels=labels)) + Lreg * 0.001

# Get predicted classes
predictions = tf.argmax(tf.nn.softmax(feat), axis=1)

# Calculate accuracy
correct_labels = tf.equal(predictions, tf.argmax(labels, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_labels, tf.float32))

In [ ]:
# Placeholder for learning rate (used in training loop)
learning_rate = tf.placeholder(tf.float32, [], name="learning_rate")

# Initialize AdamOptimizer
optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Create session
sess = tf.InteractiveSession(config=tf.ConfigProto(device_count = {'GPU': 0}))
tf.global_variables_initializer().run()

### Training loop

In [ ]:
from ipywidgets import Label, Box
from IPython.display import display

statusLabel = Label("")
display(Box([statusLabel]))

# Make batches to train
batch_size = 32
current_epoch = 0
current_learning_rate = 0.001
current_bn_decay = 0.5
current_loss = 0
current_test_accuracy = 0
num_iter = 0
start_time = time.time()
epoch_time = 0

# Try to load model file if epoch is specified
if current_epoch != 0:
    saver = tf.train.import_meta_graph("models/PointNet-%i.meta" % current_epoch)
    saver.restore(sess, tf.train.latest_checkpoint("./models"))
    current_learning_rate /= np.power(2, current_epoch // 20)
    stats = load_object("stats/PointNet_Vanilla.stats")
else:
    saver = tf.train.Saver()
    stats = { key: [] for key in ["loss", "accuracy"] }

while True:
    # If epoch of training data is complete
    if modelnet.train.is_epoch_complete():
        # Measure time
        epoch_time = time.time() - start_time
        start_time = time.time()
        
        # Avg loss
        current_loss /= num_iter
        
        # Calculate accuracy on test set
        num_iter = 0
        current_test_accuracy = 0
        epoch_complete = False
        while not modelnet.test.is_epoch_complete():
            batch_points, batch_labels = modelnet.test.next_batch(batch_size)
            current_test_accuracy += sess.run(accuracy, feed_dict={points: batch_points,
                                                                   labels: batch_labels,
                                                                   dropout_rate: 0.0})
            num_iter += 1
        current_test_accuracy /= num_iter
            
        
        # Save best model
        if current_epoch == 0 or current_test_accuracy > np.max(stats["accuracy"]):
            saver.save(sess, "models/PointNet-best")
            save_object(stats, "stats/PointNet.stats")
            
        # Increase epoch count
        current_epoch += 1
        
        # Append some stats
        stats["loss"].append(current_loss)
        stats["accuracy"].append(current_test_accuracy)
        
        # Every 20th epoch, halve the learning rate
        if current_epoch % 20 == 0:
            # Save the model
            saver.save(sess, "models/PointNet", global_step=current_epoch)
            save_object(stats, "stats/PointNet.stats")
            
            # Halve the learning rate
            current_learning_rate /= 2.0
            
            # Interpolate decay rate from 0.5 to 0.99 over 80 epochs
            current_bn_decay = lerp(0.5, 0.99, np.min([(current_epoch // 20) / 4, 1.0]))
        
        # Display training status
        statusSting  = "[Epoch %i] " % current_epoch
        statusSting += "Time: %im %is; " % (epoch_time // 60, epoch_time % 60)
        statusSting += "Loss: %f; " % current_loss
        statusSting += "Learning rate: %f; " % current_learning_rate
        statusSting += "BN decay rate: %f; " % current_bn_decay
        statusSting += "Accuracy: %f; " % current_test_accuracy
        statusSting += "--- ";
        statusSting += "Current best: [Epoch %i] " % np.argmax(stats["accuracy"])
        statusSting += "Accuracy %f; " % np.max(stats["accuracy"])
        statusLabel.value = statusSting
        
        # Reset loss
        current_loss = 0
        num_iter = 0
    
    # Get next batch
    batch_points, batch_labels = modelnet.train.next_batch(batch_size)
    
    # Rotate 10% of the time
    if np.random.rand() < 0.1:
        batch_points = rotate(batch_points, np.pi * np.random.choice([1.0 / 2.0, 1.0, 3.0 / 2.0]))
    
    # Jitter 25% of the time
    if np.random.rand() < 0.25:
        batch_points = jitter(batch_points, 0.0, 0.02)
    
    # Do training
    current_loss += sess.run([optim, loss], feed_dict={points: batch_points,
                                                       labels: batch_labels,
                                                       learning_rate: current_learning_rate,
                                                       batch_norm_decay: current_bn_decay,
                                                       dropout_rate: 0.3})[1]
    num_iter += 1

A Jupyter Widget

#### Best result
[Epoch 428] Time: 2m 53s; Loss: 0.180971; Learning rate: 0.000000; BN decay rate: 0.990000; Accuracy: 0.857372; --- Current best: [Epoch 204] Accuracy 0.870994;